# Add runtime configuration



In [1]:
import operator
from typing import Annotated, Sequence, TypedDict

from langchain_anthropic import ChatAnthropic
from langchain_core.messages import BaseMessage, HumanMessage

from langgraph.graph import END, StateGraph, START

model = ChatAnthropic(model_name="claude-2.1")


class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], operator.add]


def _call_model(state):
    response = model.invoke(state["messages"])
    return {"messages": [response]}


# Define a new graph
workflow = StateGraph(AgentState)
workflow.add_node("model", _call_model)
workflow.add_edge(START, "model")
workflow.add_edge("model", END)

app = workflow.compile()

In [2]:
app.invoke({"messages": [HumanMessage(content="hi")]})

{'messages': [HumanMessage(content='hi'),
  AIMessage(content='Hello!', response_metadata={'id': 'msg_014dq76LPex2JoaoAN1CAosP', 'model': 'claude-2.1', 'stop_reason': 'end_turn', 'stop_sequence': None, 'usage': {'input_tokens': 10, 'output_tokens': 6}}, id='run-03fd9d1c-b2aa-40be-bdad-4f7888d72a2d-0', usage_metadata={'input_tokens': 10, 'output_tokens': 6, 'total_tokens': 16})]}

In [3]:
from langchain_openai import ChatOpenAI

openai_model = ChatOpenAI()

models = {
    "anthropic": model,
    "openai": openai_model,
}


# NOTE: config在state节点上可以作为参数接收到，invoke时的config参数。
def _call_model(state, config):
    m = models[config["configurable"].get("model", "anthropic")]
    response = m.invoke(state["messages"])
    return {"messages": [response]}


# Define a new graph
workflow = StateGraph(AgentState)
workflow.add_node("model", _call_model)
workflow.add_edge(START, "model")
workflow.add_edge("model", END)

app = workflow.compile()

In [4]:
app.invoke({"messages": [HumanMessage(content="hi")]})

{'messages': [HumanMessage(content='hi'),
  AIMessage(content='Hello!', response_metadata={'id': 'msg_01EupfQ7dhFCdRGf8AUWxVFZ', 'model': 'claude-2.1', 'stop_reason': 'end_turn', 'stop_sequence': None, 'usage': {'input_tokens': 10, 'output_tokens': 6}}, id='run-6d69529a-90da-4242-a56f-ce2333a347f2-0', usage_metadata={'input_tokens': 10, 'output_tokens': 6, 'total_tokens': 16})]}

In [5]:
config = {"configurable": {"model": "openai"}}
app.invoke({"messages": [HumanMessage(content="hi")]}, config=config)

{'messages': [HumanMessage(content='hi'),
  AIMessage(content='Hello! How can I assist you today?', response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 8, 'total_tokens': 17}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-fc74b32e-863d-4d30-8581-1ed0468c8056-0', usage_metadata={'input_tokens': 8, 'output_tokens': 9, 'total_tokens': 17})]}

In [6]:
from langchain_core.messages import SystemMessage


def _call_model(state, config):
    m = models[config["configurable"].get("model", "anthropic")]
    messages = state["messages"]
    if "system_message" in config["configurable"]:
        messages = [
            SystemMessage(content=config["configurable"]["system_message"])
        ] + messages
    response = m.invoke(messages)
    return {"messages": [response]}


# Define a new graph
workflow = StateGraph(AgentState)
workflow.add_node("model", _call_model)
workflow.add_edge(START, "model")
workflow.add_edge("model", END)

app = workflow.compile()

In [7]:
app.invoke({"messages": [HumanMessage(content="hi")]})

{'messages': [HumanMessage(content='hi'),
  AIMessage(content='Hello!', response_metadata={'id': 'msg_01D76XLWSL4RFQeF86nRq6et', 'model': 'claude-2.1', 'stop_reason': 'end_turn', 'stop_sequence': None, 'usage': {'input_tokens': 10, 'output_tokens': 6}}, id='run-ed052761-e542-4666-84ea-9c8a128d0b12-0', usage_metadata={'input_tokens': 10, 'output_tokens': 6, 'total_tokens': 16})]}

In [8]:
config = {"configurable": {"system_message": "respond in italian"}}
app.invoke({"messages": [HumanMessage(content="hi")]}, config=config)

{'messages': [HumanMessage(content='hi'),
  AIMessage(content='Ciao!', response_metadata={'id': 'msg_01J1sCnkbb3ojYqsENk8Mr2W', 'model': 'claude-2.1', 'stop_reason': 'end_turn', 'stop_sequence': None, 'usage': {'input_tokens': 14, 'output_tokens': 7}}, id='run-b7c83ed3-c432-4336-a1d5-be2d92ae204c-0', usage_metadata={'input_tokens': 14, 'output_tokens': 7, 'total_tokens': 21})]}